<a href="https://colab.research.google.com/github/doaadoukh/imlo_final/blob/main/final_imlo_coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import PyTorch packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split

#Transformations for training and test sets
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#Loading the datasets
flowers102_dataset = datasets.Flowers102(root='data', split='train', download=True, transform=train_transform)
test_dataset = datasets.Flowers102(root='data', split='test', download=True, transform=test_transform)

#Splitting dataset into training, validation (80-20)
dataset_size = len(flowers102_dataset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size
train_dataset, val_dataset = random_split(flowers102_dataset, [train_size, val_size])

#Batch size
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

#Defining the CNN with dropout rate
class Flower_Classifier_CNN(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(Flower_Classifier_CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(512 * 14 * 14,1024)
        self.fc2 = nn.Linear(1024,102)

    def forward(self,x):
      x = self.pool(F.relu(self.bn1(self.conv1(x))))
      x = self.pool(F.relu(self.bn2(self.conv2(x))))
      x = self.pool(F.relu(self.bn3(self.conv3(x))))
      x = self.pool(F.relu(self.bn4(self.conv4(x))))
      x = x.view(-1, 512 * 14 * 14)
      x = self.dropout(F.relu(self.fc1(x)))
      x = self.fc2(x)
      return x

#Setting the hyperparameters
learning_rate = 0.0005
weight_decay = 0.01
dropout_rate = 0.5
no_epochs = 2000

#Defining the loss function and the optimiser
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Flower_Classifier_CNN(dropout_rate=dropout_rate).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

#Cosine annealing learning rate scheduler
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

#Functiona for calculating accuracy
def calculate_acc(loader,model):
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for inputs, labels in loader:
      inputs = inputs.to(device)
      labels = labels.to(device)
      outputs = model(inputs)
      _, preds = torch.max(outputs,1)
      correct += torch.sum(preds == labels.data)
      total += labels.size(0)
  return(correct.double()/total)*100


#Function to train the model
def train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, scheduler, no_epochs=2000):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    for epoch in range(no_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)
            total += labels.size(0)

        scheduler.step()
        epoch_loss = running_loss / len(train_loader.dataset)
        train_accuracy = (correct.double()/total)*100
        val_accuracy = calculate_acc(val_loader,model)

        if epoch % 50 == 0:
          print(f'Epoch {epoch+1}/{no_epochs}, Batch Size: {batch_size}, Loss: {epoch_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Acc: {val_accuracy:.2f}%')

    print('Training is now complete')
    return model

#Training the model
trained_model = train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, scheduler, no_epochs)

#Evaluating the model on the test set
def eval_model(model, test_loader):
  test_acc = calculate_acc(test_loader,model)
  print(f'Test Accuracy: {test_acc: .2f}%')

#Evaluating the training model
eval_model(trained_model, test_loader)



Epoch 1/2000, Batch Size: 64, Loss: 0.3352, Train Acc: 1.23%, Val Acc: 3.43%
Epoch 51/2000, Batch Size: 64, Loss: 0.0684, Train Acc: 2.21%, Val Acc: 1.96%
Epoch 101/2000, Batch Size: 64, Loss: 0.0679, Train Acc: 2.82%, Val Acc: 1.96%
Epoch 151/2000, Batch Size: 64, Loss: 0.0657, Train Acc: 3.68%, Val Acc: 1.47%
Epoch 201/2000, Batch Size: 64, Loss: 0.0595, Train Acc: 12.01%, Val Acc: 6.86%
Epoch 251/2000, Batch Size: 64, Loss: 0.0519, Train Acc: 16.67%, Val Acc: 13.24%
Epoch 301/2000, Batch Size: 64, Loss: 0.0465, Train Acc: 25.86%, Val Acc: 13.24%
Epoch 351/2000, Batch Size: 64, Loss: 0.0487, Train Acc: 22.43%, Val Acc: 11.27%
Epoch 401/2000, Batch Size: 64, Loss: 0.0438, Train Acc: 29.53%, Val Acc: 20.10%
Epoch 451/2000, Batch Size: 64, Loss: 0.0383, Train Acc: 37.25%, Val Acc: 21.57%
Epoch 501/2000, Batch Size: 64, Loss: 0.0340, Train Acc: 41.18%, Val Acc: 27.94%
Epoch 551/2000, Batch Size: 64, Loss: 0.0298, Train Acc: 49.88%, Val Acc: 34.31%
Epoch 601/2000, Batch Size: 64, Loss: 0.